In [2]:
import cbpro
import os
import pickle
import time
import pandas as pd
import numpy as np
import datetime as dt

# Authentication credentials
api_key = os.environ.get('coinbaseSandboxKey')
api_secret = os.environ.get('coinbaseSandboxSecret')
passphrase = os.environ.get('coinbaseSandboxPassphrase')

# sandbox authenticated client
auth_client = cbpro.AuthenticatedClient(api_key, api_secret, passphrase, \
                                        api_url='https://api-public.sandbox.pro.coinbase.com')
# live account authenticated client
# uses a different set of API access credentials (api_key, api_secret, passphrase)
# auth_client = cbpro.AuthenticatedCliet(api_key, api_secret, passphrase)

# get_orders generator:
orders_gen = auth_client.get_orders()

# trading algorithm
sel = ['tradeId', 'amountK', 'currency', 'grossPL', 'isBuy']

log_file = 'automated_strategy.log'

# loads the persisted trading algorithm object
algorithm = pickle.load(open('algorithm_dailyBTC.pkl', 'rb'))

def logger_monitor(message, time=True, sep=True):
    # logger and monitor function
    with open(log_file, 'a') as f:
        t = str(dt.datetime.now())
        msg = ''
        if time:
            msg += '\n' + t + '\n'
        if sep:
            msg += 66 * '=' + '\n'
        msg += message + '\n\n'
        # displays message to screen
        print('Trading Log Output:'+'\n')
        print(msg)
        # writes the message to the log file
        f.write(msg)

def report_positions(pos):
    '''Prints, logs, and sends position data'''
    out = '\n\n' + 50 * '=' + '\n'
    out += 'Going {}.\n'.format(pos) + '\n'
    time.sleep(2) # waits for the order to be executed
    # get orders (will possibly make multiple HTTP requests)
    get_orders_gen = auth_client.get_orders()
    get_orders = list(get_orders_gen)
    out += str(get_orders) + '\n'
    out += 50 * '=' + '/n'
    logger_monitor(out)
    print(out)

# callback function to implement the trading strategy

def automated_strategy(data, dataframe):
    global min_bars, position, df
    ldf = len(dataframe)
    df = dataframe.resample(bar, label='right').last().ffill()
    if ldf % 20 == 0:
        print('%3d' % len(dataframe), end=',')
    
    if len(df) > min_bars:
        min_bars = len(df)
        df['Mid'] = df[['best_bid', 'best_ask']].mean(axis=1)
        df['Returns'] = np.log(df['Mid']/df['Mid'].shift(1))
        df['Direction'] = np.where(df['Returns'] > 0, 1, -1)
        features = df['Direction'].iloc[-(lags + 1): -1]
        features = features.values.reshape(1, -1)
        signal = algorithm.predict(features)[0]
        
        # logs and sends major financial information
        logger_monitor('MOST RECENT DATA\n'+
                      str(df[['Mid', 'Returns', 'Direction']].tail()),
                      False)
        logger_monitor('features: ' + str(features) + '\n' +
                      'position:  ' + str(position) + '\n' +
                      'signal:    ' + str(signal), False)
        
        # trading logic
        
        if position in [0, -1] and signal == 1:
            auth_client.place_market_order(product_id = symbol, 
                               side = 'buy', 
                               funds = amount - position * amount)
            position = 1
            report_positions('LONG')
            
        elif position in [0, 1] and signal == -1:
            auth_client.place_market_order(product_id = symbol,
                                          side = 'sell',
                                          funds = amount + position * amount)
            position = -1
            report_positions('SHORT')
        else: # no trade
            logger_monitor('no trade placed')
            
        logger_monitor('****END OF CYCLE****\n\n', False, False)
            
    if len(dataframe) > max:
        print('ending trading, max # ticks received')
        # close market data web socket client
        wsClient.close()
        exit
        

if __name__ == '__main__':
    import sys
    import cbpro
    import time
    
    # parameters for the trading algorithm
    symbol = 'BTC-USD'
    bar = '15s'         # 15s is for testing; reset to trading frequency
    amount = 100        # amount to be traded in $USD
    position = 0        # initial, neutral, position
    lags = 3            # number of lags for features data
    max = 6            # number of ticks to receive

    # minumum number of resampled bars required for the first predicted value (& first trade)
    min_bars = lags + 1 

    df = pd.DataFrame()
    data = pd.DataFrame()

    # define on_message behavior of the web socket client
    #class MyWebsocketClient(cbpro.WebsocketClient):
    #    def on_message(self, msg):
    #        data = data.append(msg)
    #        if len(data) >= max:
    #            data.to_csv('data_file.csv')

    # the main asynchronous loop using the callback function
    # Coinbase Pro web socket connection is rate-limited to 4 seconds per request per IP.
    wsClient = cbpro.WebsocketClient(url="wss://ws-feed.pro.coinbase.com", products=symbol, channels=["ticker"], \
                                     should_print=False)
    #wsClient = MyWebsocketClient(url="wss://ws-feed.pro.coinbase.com", products=symbol, channels=["ticker"])
    
    wsClient.start()
    
    try:
        wsClient.on_message((automated_strategy,))
    except:
        print('Error - stopping program')
        wsClient.close()